In [1]:
# Se importan todas las librerías que vamos a necesitar
##'''Importing Libraries'''
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, recall_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import preprocessor as p
from keras.preprocessing.text import text_to_word_sequence   ##NEW
from nltk.stem import WordNetLemmatizer                      ##NEW
from pywsd.utils import lemmatize_sentence
from sklearn import svm

Using TensorFlow backend.
Warming up PyWSD (takes ~10 secs)... took 17.020800352096558 secs.


In [2]:
import sys

In [3]:
sys.executable

'/home/dsc/anaconda3/envs/tfm/bin/python'

In [4]:
sys.path

['/home/dsc/anaconda3/envs/tfm/lib/python36.zip',
 '/home/dsc/anaconda3/envs/tfm/lib/python3.6',
 '/home/dsc/anaconda3/envs/tfm/lib/python3.6/lib-dynload',
 '',
 '/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages',
 '/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages/IPython/extensions',
 '/home/dsc/.ipython']

In [5]:
from jupyter_core.paths import jupyter_data_dir
print(jupyter_data_dir())

/home/dsc/.local/share/jupyter


In [6]:
# Se importa el dataset y lo guardamos en 'data'. Se guardan solamente el texto de los tweets en 'tweets'
##data= pd.read_csv('Downloads/Tweets.csv')
data= pd.read_csv('Tweets.csv')
tweets = data['text']
tweets.head()

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
Name: text, dtype: object

In [7]:
# Se cambian las variables categóricas de la columna airline_sentiment a variables numéricas: 
## Negative : 0
## Neutral : 1
## Positive : 2

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['target'] = lb.fit_transform(data['airline_sentiment'])
data[['airline_sentiment', 'target']]
target = data['target']
print(data[['airline_sentiment', 'target']])

      airline_sentiment  target
0               neutral       1
1              positive       2
2               neutral       1
3              negative       0
4              negative       0
5              negative       0
6              positive       2
7               neutral       1
8              positive       2
9              positive       2
10              neutral       1
11             positive       2
12             positive       2
13             positive       2
14             positive       2
15             negative       0
16             positive       2
17             negative       0
18             positive       2
19             positive       2
20             negative       0
21             positive       2
22             positive       2
23              neutral       1
24             negative       0
25             negative       0
26             negative       0
27              neutral       1
28             negative       0
29              neutral       1
...     

In [8]:
# Se comprueba la correlación entre el tweet (text) y su etiqueta
df = data[['text', 'target']]
df.sample(4)

,text,target
8995,@USAirways I forgot my password and I don't kn...,1
11920,@AmericanAir still waiting for a flight... I s...,0
6977,“@JetBlue: Our fleet's on fleek. http://t.co/t...,1
1781,@united reservation was made last July. I want...,0


In [9]:
# Se comprueba que el campo de tweets y el campo de target tiene la misma longitud.
print('Número de tweets: ' + ' ' + str(len(tweets)),'\nNúmero de targets:' + ' ' + str(len(target)))

Número de tweets:  14640 
Número de targets: 14640


In [10]:
# Se comprueba que codifica correctamente
# Al final de la tabla se observa la nueva variable (target)
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,target
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),1
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),2
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),1
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0


In [11]:
##OLD..
# Se preprocesan los tweets con tweet-preprocessor5.0 y se eliminan solamente las menciones y los hashtags.
# Se guardan los tweets en un lista, la cual contiene todos los tweets sin hashtags ni menciones.
#'''Preprocessing : tweet preprocessor 5.0'''
#tweets_preprocessed = []
#for tweet in tweets:
#    p.set_options(p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.URL)
#    tweets_preprocessed.append(p.clean(tweet))
    

In [12]:
##tweets_preprocessed

In [13]:
print(tweets)

0                      @VirginAmerica What @dhepburn said.
1        @VirginAmerica plus you've added commercials t...
2        @VirginAmerica I didn't today... Must mean I n...
3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
5        @VirginAmerica seriously would pay $30 a fligh...
6        @VirginAmerica yes, nearly every time I fly VX...
7        @VirginAmerica Really missed a prime opportuni...
8          @virginamerica Well, I didn't…but NOW I DO! :-D
9        @VirginAmerica it was amazing, and arrived an ...
10       @VirginAmerica did you know that suicide is th...
11       @VirginAmerica I &lt;3 pretty graphics. so muc...
12       @VirginAmerica This is such a great deal! Alre...
13       @VirginAmerica @virginmedia I'm flying your #f...
14                                  @VirginAmerica Thanks!
15           @VirginAmerica SFO-PDX schedule is still MIA.
16       @VirginAmerica So excited for my first cross c.

In [14]:
# apostrophe lookup dict
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [10]:
###NO EJECUTAR!! ES EJEMPLO DE COMO TESTEARLO SOLO CON UNA FRASE!!

lemmatizer = WordNetLemmatizer()
import emoji

test_str = "Thank you @VirginAmerica for you amazing 👍 #virginamerica"

def clean_text(text):
    text = re.sub(r'(?:@[\w_]+)', " ", text) # @-mentions
    text = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", text) # hash-tags
    text = emoji.demojize(text).replace('_','')
    #text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w{1,1}\b', '', text)
    #text = re.sub(r'http.?://[^\s]+[\s]?', '', text) #Remove URLs
    return text

test_str = clean_text(test_str)

def cleanupText(s):
    stopset = set(stopwords.words('english'))
    stopset.add('wikipedia')

    tokens =sequence=text_to_word_sequence(s, 
                                        filters="\"!'#$%&()*+,-˚˙./:;‘“<=·>?@[]^_`{|}~\t\n",
                                        lower=True,
                                        split=" ")
    tokens=[APPO[token] if token in APPO else token for token in tokens]
    for token in tokens:
        lemmatizer.lemmatize(token, 'v')
    cleanup = " ".join(filter(lambda word: word not in stopset, tokens))
    return cleanup

print(cleanupText(test_str))

thank amazing thumbsup customer support team tuesday 11 28 returning lost bag less 24h


In [15]:
print(tweets)

0                      @VirginAmerica What @dhepburn said.
1        @VirginAmerica plus you've added commercials t...
2        @VirginAmerica I didn't today... Must mean I n...
3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
5        @VirginAmerica seriously would pay $30 a fligh...
6        @VirginAmerica yes, nearly every time I fly VX...
7        @VirginAmerica Really missed a prime opportuni...
8          @virginamerica Well, I didn't…but NOW I DO! :-D
9        @VirginAmerica it was amazing, and arrived an ...
10       @VirginAmerica did you know that suicide is th...
11       @VirginAmerica I &lt;3 pretty graphics. so muc...
12       @VirginAmerica This is such a great deal! Alre...
13       @VirginAmerica @virginmedia I'm flying your #f...
14                                  @VirginAmerica Thanks!
15           @VirginAmerica SFO-PDX schedule is still MIA.
16       @VirginAmerica So excited for my first cross c.

In [23]:
lemmatizer = WordNetLemmatizer()
import emoji

tweets_cleaned = []  

for tweet in tweets:
    tweet = re.sub(r'(?:@[\w_]+)', " ", tweet) # @-mentions
    tweet = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", tweet) # hash-tags
    tweet = emoji.demojize(tweet).replace('_','')
   #text = re.sub(r'\d+', '', text)
    tweet = re.sub(r'\b\w{1,1}\b', '', tweet)
   #text = re.sub(r'http.?://[^\s]+[\s]?', '', text) #Remove URLs
    #return text
    tweets_cleaned.append(tweet) 
##test_str = clean_text(test_str)
##tweets_preprocessed = clean_text(test_str)


In [24]:
tweets_cleaned

['  What   said.',
 "  plus you've added commercials to the experience... tacky.",
 "   didn' today... Must mean  need to take another trip!",
 '  it\' really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
 "  and it'  really big bad thing about it",
 "  seriously would pay $30  flight for seats that didn' have this playing.\nit' really the only bad thing about flying VA",
 '  yes, nearly every time  fly VX this “ear worm” won’ go away :)',
 '  Really missed  prime opportunity for Men Without Hats parody, there. https://.co/mWpG7grEZP',
 "  Well,  didn'…but NOW  DO! :-",
 "  it was amazing, and arrived an hour early. You're too good to me.",
 '  did you know that suicide is the second leading cause of death among teens 10-24',
 '   &lt; pretty graphics. so much better than minimal iconography. :',
 "  This is such  great deal! Already thinking about my 2nd trip to   &amp;  haven' even gone on my 1st trip yet! ;",
 "    ' flying yo

In [25]:
##ADAPTARLO PARA QUE FUNCIONE CON UN FOR PARA TODO EL CAMPO.
tweets_cleaned2 = []

#def cleanupText(s):
for sentence in tweets_cleaned:
    sentence = set(stopwords.words('english'))
    sentence.add('wikipedia')

    tokens =sequence=text_to_word_sequence(sentence) 
                                       ## filters="\"!'#$%&()*+,-˚˙./:;‘“<=·>?@[]^_`{|}~\t\n")
                                        ##lower=True,
                                        ##split=" ")
    tokens=[APPO[token] if token in APPO else token for token in tokens]
    for token in tokens:
        lemmatizer.lemmatize(token, 'v')
    ##cleanup = " ".join(filter(lambda word: word not in stopset, tokens))
    lem = " ".join(filter(lambda word: word not in stopset, tokens))
    tweets_cleaned_2.append(lem)
    ##return cleanup

##print(cleanupText(test_str))

AttributeError: 'set' object has no attribute 'lower'

In [20]:
## TEST 
lemmatizer = WordNetLemmatizer()
import emoji

#######################  TEST  #########################
##'I ❤️ flying . ☺️👍'
## 'Very nicely done. 👏',
## "hi! I just bked a cool birthday trip with you, but i can't add my elevate no. cause i entered my middle name during Flight Booking Problems 😢"

###EXEMPLE How to transform  👍 in  :thumbsup:
test_str = "Thank you @Delta for you amazing 👍 "

#######################################################

def clean_text(text):
    text = re.sub(r'(?:@[\w_]+)', " ", text) # @-mentions
    text = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", text) # hash-tags
    text = emoji.demojize(text).replace('_','')
    #text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w{1,1}\b', '', text)
    #text = re.sub(r'http.?://[^\s]+[\s]?', '', text) #Remove URLs
    return text

test_str = clean_text(test_str)
print(test_str)

Thank you   for you amazing :thumbsup: 


In [21]:
##Ejemplo de como se limpia el texto del ejemplo superior!!
def cleanupText(s):
    stopset = set(stopwords.words('english'))
    stopset.add('wikipedia')

    tokens =sequence=text_to_word_sequence(s, 
                                        filters="\"!'#$%&()*+,-˚˙./:;‘“<=·>?@[]^_`{|}~\t\n",
                                        lower=True,
                                        split=" ")
    tokens=[APPO[token] if token in APPO else token for token in tokens]
    for token in tokens:
        lemmatizer.lemmatize(token, 'v')
    cleanup = " ".join(filter(lambda word: word not in stopset, tokens))
    return cleanup

print(cleanupText(test_str))

thank amazing thumbsup


In [20]:
##for tweet in tweets:
  ##  print(tweets_cleaned)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['  What   said.', "  plus you've added commercials to the experience... tacky.", "   didn' today... Must mean  need to take another trip!", '  it\' really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse', "  and it'  really big bad thing about it", "  seriously would pay $30  flight for seats that didn' have this playing.\nit' really the only bad thing about flying VA", '  yes, nearly every time  fly VX this “ear worm” won’ go away :)', '  Really missed  prime opportunity for Men Without Hats parody, there. https://.co/mWpG7grEZP', "  Well,  didn'…but NOW  DO! :-", "  it was amazing, and arrived an hour early. You're too good to me.", '  did you know that suicide is the second leading cause of death among teens 10-24', '   &lt; pretty graphics. so much better than minimal iconography. :', "  This is such  great deal! Already thinking about my 2nd trip to   &amp;  haven' even gone on my 1st trip yet! ;", "    ' flying your     skies 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-2ac21dd352be>", line 2, in <module>
    print(tweets_cleaned)
  File "/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages/ipykernel/iostream.py", line 383, in write
    if self.echo is not None:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/envs/tfm/lib/python3.6/site-packages/IPython/core/ultratb.py

KeyboardInterrupt: 

In [39]:
def cleanupText(s):
    stopset = set(stopwords.words('english'))
    stopset.add('wikipedia')

    tokens =sequence=text_to_word_sequence(s, 
                                        filters="\"!'#$%&()*+,-˚˙./:;‘“<=·>?@[]^_`{|}~\t\n",
                                        lower=True,
                                        split=" ")
    tokens=[APPO[token] if token in APPO else token for token in tokens]
    for token in tokens:
        lemmatizer.lemmatize(token, 'v')
    cleanup = " ".join(filter(lambda word: word not in stopset, tokens))
    return cleanup

print(cleanupText(test_str))

thank amazing thumbsup customer support team tuesday 11 28 returning lost bag less 24h
